# PKI Secrets Engine - Build Your Own CA

Vault's PKI secrets engine can dynamically generate X.509 certificates on demand. This allows services to acquire certificates without going through the usual manual process of generating a private key and Certificate Signing Request (CSR), submitting to a Certificate Authority (CA), and then waiting for the verification and signing process to complete.
<img align=center src=Images/pki-vault-as-rootCA.png width=600/>

Challenge: 
The Traditional PKI process workflow takes a long time, which motivates organizations to create certificates which do not expire for a year or more.

Solution:
Use Vault to create X.509 certificates simply.

## Setting Up Vault Client

In [1]:
export VAULT_ADDR=http://localhost:8200
export VAULT_TOKEN=root
vault login root

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       87H5mcaZNC4PPYDct163lx0c
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [2]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.9.2
Storage Type    inmem
Cluster Name    vault-cluster-4ef336c9
Cluster ID      948de1b6-56e7-fdb5-5fca-f59ec7e48b49
HA Enabled      false


---
## Set Up PKI Secrets Engine

### Step#1: Set up PKI Secrets Engine

In [3]:
vault secrets enable pki

Success! Enabled the pki secrets engine at: pki/


In [4]:
#Increase the TTL by tuning the secrets engine. 
vault secrets tune -max-lease-ttl=8760h pki

Success! Tuned the secrets engine at: pki/


In [5]:
vault secrets list

Path                          Type         Accessor              Description
----                          ----         --------              -----------
cubbyhole/                    cubbyhole    cubbyhole_a5e84643    per-token private secret storage
identity/                     identity     identity_3f771378     identity store
openldap/                     openldap     openldap_daf63dde     n/a
pki/                          pki          pki_7bc04361          n/a
secret/                       kv           kv_8031e0df           key/value secret storage
ssh-client-signe-non-prod/    ssh          ssh_a4138960          n/a
ssh-client-signer/            ssh          ssh_bbe10c96          n/a
ssh-user-ca/                  ssh          ssh_ae95f83c          n/a
sys/                          system       system_5ec4bca7       system endpoints used for control, policy and debugging


### Step#2: Generate Root CA

In [6]:
#Creates a root certificate valid for 10 years
vault write -field=certificate pki/root/generate/internal \
     common_name="example.com" \
     issuer_name="root-2022" \
     ttl=87600h > root_2022_ca.crt

In [7]:
#List the Issuer informatiom for the Root CA
vault list pki/issuers/

No value found at pki/issuers


: 2

In [8]:
vault write pki/roles/2022-servers allow_any_name=true

Success! Data written to: pki/roles/2022-servers


In [9]:
vault write pki/config/urls \
     issuing_certificates="$VAULT_ADDR/v1/pki/ca" \
     crl_distribution_points="$VAULT_ADDR/v1/pki/crl"

Success! Data written to: pki/config/urls


### Step#3: Generate Intermediate CA

In [10]:
vault secrets enable -path=pki_int pki

Success! Enabled the pki secrets engine at: pki_int/


In [11]:
vault secrets tune -max-lease-ttl=43800h pki_int

Success! Tuned the secrets engine at: pki_int/


In [12]:
vault write -format=json pki_int/intermediate/generate/internal \
     common_name="example.com Intermediate Authority" \
     issuer_name="example-dot-com-intermediate" \
     | jq -r '.data.csr' > pki_intermediate.csr


In [13]:
vault write -format=json pki/root/sign-intermediate \
     issuer_ref="root-2022" \
     csr=@pki_intermediate.csr \
     format=pem_bundle ttl="43800h" \
     | jq -r '.data.certificate' > intermediate.cert.pem


In [14]:
vault write pki_int/intermediate/set-signed certificate=@intermediate.cert.pem

Success! Data written to: pki_int/intermediate/set-signed


### Step#3: Create a role
##### A role is a logical name that maps to a policy used to generate those credentials. It allows configuration parameters to control certificate common names, alternate names, the key uses that they are valid for, and more.

In [15]:
vault write pki_int/roles/example-dot-com \
     issuer_ref="$(vault read -field=default pki_int/config/issuers)" \
     allowed_domains="example.com" \
     allow_subdomains=true \
     max_ttl="720h"

No value found at pki_int/config/issuers
Success! Data written to: pki_int/roles/example-dot-com


### Step#4: Request a leaf certificate using Vault

In [22]:
vault write pki_int/issue/example-dot-com common_name="test.example.com" ttl="24h"

Key                 Value
---                 -----
ca_chain            [-----BEGIN CERTIFICATE-----
MIIDpjCCAo6gAwIBAgIUHORh+vziM76+VPyr76CNCowYcIAwDQYJKoZIhvcNAQEL
BQAwFjEUMBIGA1UEAxMLZXhhbXBsZS5jb20wHhcNMjIwOTIxMDE0MDI4WhcNMjMw
OTIxMDE0MDU4WjAtMSswKQYDVQQDEyJleGFtcGxlLmNvbSBJbnRlcm1lZGlhdGUg
QXV0aG9yaXR5MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA8rd/vMSX
J5yIiHjqnFMuSqpQjEiri4Z7VwLao5aPzM8VVtvo/4MJCeidHrfRJP/qb0DNmYNX
o4zNjMwtTep023lByqkfSa+oVXlu62w7UZo/tAIp83qUxIqJaFXeomu76hJI9ZrF
Wii7fAt/XOcizQ05b9Uegofs9LyPVOJYcMI/uo1Hng+TfWHaEtDiG19jOTocpsWO
9RQgDuWumoU9WGQWK3Xjv6HxgwaApCTjVd+I6RduhKX4Fgu5UTlJG6Dhsk0lzvn/
w6MtqO3XPw74bLJrzPferj2Raf0EbRwpyHAJCziRkFL1ctzUpRoUntYXAlXeEypu
HFZAb5zW8DNMkwIDAQABo4HUMIHRMA4GA1UdDwEB/wQEAwIBBjAPBgNVHRMBAf8E
BTADAQH/MB0GA1UdDgQWBBQ8gBjhhoY+ondoPMKT+U4lM89BazAfBgNVHSMEGDAW
gBTzIxn9TA+q0n6hPHq73m830enoHTA7BggrBgEFBQcBAQQvMC0wKwYIKwYBBQUH
MAKGH2h0dHA6Ly9sb2NhbGhvc3Q6ODIwMC92MS9wa2kvY2EwMQYDVR0fBCowKDAm
oCSgIoYgaHR0cDovL2xvY2FsaG9zdDo4MjAwL3YxL3BraS9jcmwwDQ

In [23]:
curl --header "X-Vault-Token: $VAULT_TOKEN" \
    --request POST \
    --data '{"common_name": "api.example.com", "ttl": "24h"}' \
    $VAULT_ADDR/v1/pki_int/issue/example-dot-com | jq


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6048    0  6000  100    48  14527    116 --:--:-- --:--:-- --:--:-- 14644
{
  "request_id": "d4cee8d7-6a0d-819d-e531-6baa5c8327d5",
  "lease_id": "",
  "renewable": false,
  "lease_duration": 0,
  "data": {
    "ca_chain": [
      "-----BEGIN CERTIFICATE-----\nMIIDpjCCAo6gAwIBAgIUHORh+vziM76+VPyr76CNCowYcIAwDQYJKoZIhvcNAQEL\nBQAwFjEUMBIGA1UEAxMLZXhhbXBsZS5jb20wHhcNMjIwOTIxMDE0MDI4WhcNMjMw\nOTIxMDE0MDU4WjAtMSswKQYDVQQDEyJleGFtcGxlLmNvbSBJbnRlcm1lZGlhdGUg\nQXV0aG9yaXR5MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA8rd/vMSX\nJ5yIiHjqnFMuSqpQjEiri4Z7VwLao5aPzM8VVtvo/4MJCeidHrfRJP/qb0DNmYNX\no4zNjMwtTep023lByqkfSa+oVXlu62w7UZo/tAIp83qUxIqJaFXeomu76hJI9ZrF\nWii7fAt/XOcizQ05b9Uegofs9LyPVOJYcMI/uo1Hng+TfWHaEtDiG19jOTocpsWO\n9RQgDuWumoU9WGQWK3Xjv6HxgwaApCTjVd+I6RduhKX4Fgu5UTlJG6Dhsk0lzvn/\nw6MtqO3XPw74bLJrzPferj2Raf0EbRwpyHAJCziRkFL1ctzU

In [18]:
curl --header "X-Vault-Token: $VAULT_TOKEN" \
    --request POST \
    --data '{"common_name": "test1.tcb.com", "ttl": "24h"}' \
    $VAULT_ADDR/v1/pki_int/issue/example-dot-com | jq


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   112  100    66  100    46  22000  15333 --:--:-- --:--:-- --:--:-- 37333
{
  "errors": [
    "common name test1.tcb.com not allowed by this role"
  ]
}


In [19]:
results=$(vault write pki_int/issue/example-dot-com common_name="test3.example.com" ttl="24h" -format=json)
parsed=$(echo $results | jq .data -r)
echo $parsed | jq .certificate -r > leaf-cert-3.pem
echo $parsed | jq .private_key -r > leaf-cert-3-key.pem

In [20]:
cat leaf-cert-3.pem
cat leaf-cert-3-key.pem

-----BEGIN CERTIFICATE-----
MIIDaDCCAlCgAwIBAgIUYLJavBvjaooWYQDYUqtv5OOaI+YwDQYJKoZIhvcNAQEL
BQAwLTErMCkGA1UEAxMiZXhhbXBsZS5jb20gSW50ZXJtZWRpYXRlIEF1dGhvcml0
eTAeFw0yMjA5MjEwMTQwMzZaFw0yMjA5MjIwMTQxMDZaMBwxGjAYBgNVBAMTEXRl
c3QzLmV4YW1wbGUuY29tMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA
oM6fnMjEyTQRjb9lhOIuFMOvorZcj2QrM3yvztm20oarxir/XdNB9vagsCgLlr1K
usXkR7+6hrCgcnekyvVVEWbIOz7EID2OZZ3o0+LV+xUNK0Hp94N0CNMm1rdLaiFa
Dk7yNGNFgUPRJ4PmC2M/48M8hkxA72InLaBrK+wPSi3fpZl+eMpmfQDMidDDF5yw
EfVFbNAq5z6DfHNDbroM9KcgTdgA5HeIRgzOLE4IOVMVJGSeNJ2Xaub9P3LRvY4P
iAKKGVJxIXGngcXWq/KR5Nh3lcorRZyobu3e0JrIWd0jvkZAxj4de3ovuk3xMzTc
lsgAe6DImqiKaySx1bk2HQIDAQABo4GQMIGNMA4GA1UdDwEB/wQEAwIDqDAdBgNV
HSUEFjAUBggrBgEFBQcDAQYIKwYBBQUHAwIwHQYDVR0OBBYEFA+lmSu95JcxT92h
uEtJJbPp2K7JMB8GA1UdIwQYMBaAFDyAGOGGhj6id2g8wpP5TiUzz0FrMBwGA1Ud
EQQVMBOCEXRlc3QzLmV4YW1wbGUuY29tMA0GCSqGSIb3DQEBCwUAA4IBAQDSxum+
I+3PGvYIZrDWGeczq/h1KBnMl+HPDyVChRPOIvPo3eik209FsKCvb6/MNziqeKkZ
5+E9N2hdE4QnzQEMZVtnctIx0AJ8U232tq+93A1bhgWTT3JqGFLTArFT3Pp8sv

In [21]:
openssl x509 -in  leaf-cert-3.pem -text -noout

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            60:b2:5a:bc:1b:e3:6a:8a:16:61:00:d8:52:ab:6f:e4:e3:9a:23:e6
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: CN = example.com Intermediate Authority
        Validity
            Not Before: Sep 21 01:40:36 2022 GMT
            Not After : Sep 22 01:41:06 2022 GMT
        Subject: CN = test3.example.com
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                RSA Public-Key: (2048 bit)
                Modulus:
                    00:a0:ce:9f:9c:c8:c4:c9:34:11:8d:bf:65:84:e2:
                    2e:14:c3:af:a2:b6:5c:8f:64:2b:33:7c:af:ce:d9:
                    b6:d2:86:ab:c6:2a:ff:5d:d3:41:f6:f6:a0:b0:28:
                    0b:96:bd:4a:ba:c5:e4:47:bf:ba:86:b0:a0:72:77:
                    a4:ca:f5:55:11:66:c8:3b:3e:c4:20:3d:8e:65:9d:
                    e8:d3:e2:d5:fb:15:0d:2b:41:e9:f7:83:74:08:d3:
                    26:d6:b7:4b:6a:21:5a:0e:4e:f

---
## Revoke certificates

##### If a certificate must be revoked, you can easily perform the revocation action which will cause the CRL to be regenerated. When the CRL is regenerated, any expired certificates are removed from the CRL.

In [21]:
vault write pki_int/revoke serial_number='38:11:88:df:be:8c:cb:a1:90:c6:01:88:5f:94:4b:58:fe:f2:42:5a'

Key                        Value
---                        -----
revocation_time            1663670667
revocation_time_rfc3339    2022-09-20T10:44:27.733219Z


---
## Remove Expired Certificates

##### Keep the storage backend and CRL by periodically removing certificates that have expired and are past a certain buffer period beyond their expiration time.

In [22]:
vault write pki/tidy tidy_cert_store=true tidy_revoked_certs=true

WARNING! The following warnings were returned from Vault:

  * Tidy operation successfully started. Any information from the operation
  will be printed to Vault's server logs.



---